In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from taskdataset import TaskDataset


In [11]:
dataset = torch.load("data/SybilAttack.pt")

print(dataset.ids, dataset.imgs, dataset.labels)


[101031, 8526, 43127, 191394, 298792, 121086, 149475, 102605, 163605, 101855, 135720, 202472, 78948, 205516, 130325, 61352, 12573, 148600, 288522, 121039, 142636, 248729, 98173, 110229, 251315, 294900, 69630, 246410, 205724, 241323, 239570, 74090, 206834, 248228, 277825, 177638, 178460, 195306, 57873, 239078, 142185, 240902, 77409, 114516, 158823, 257739, 162041, 19492, 105759, 300664, 233603, 213129, 28202, 154881, 56113, 218900, 77103, 166599, 97556, 73616, 236198, 166840, 77532, 3669, 191063, 5040, 57826, 74718, 304774, 48059, 21290, 268421, 236305, 229322, 280199, 305084, 107778, 230929, 272095, 141814, 297494, 262306, 7149, 222224, 66291, 104205, 252405, 231398, 45372, 292658, 144859, 201391, 1212, 270096, 287366, 208108, 71574, 260196, 210798, 162229, 292402, 170376, 193023, 6021, 135462, 122707, 298246, 171671, 263538, 76182, 141419, 256928, 293835, 13331, 217190, 19693, 122601, 187403, 35388, 189706, 56506, 130137, 51870, 41849, 81127, 88318, 277129, 254901, 100903, 100569, 195

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
class Linear(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        return x

In [ ]:
l1_penalty = l1_weight * sum([p.abs().sum() for p in net.hidden.parameters()])